In [1]:
%matplotlib inline
%precision 16
import numpy as np
import matplotlib.pyplot as plt
import scipy.linalg as sl

# prints in a way that we can cut and paste directly into code
from pprint import pprint

# font sizes for plots
plt.rcParams['font.size'] = 12
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['Arial', 'Dejavu Sans']

# Gaussian Elimination

## Steps
Step 1 : Elimination Stage \
Step 2 : Substitution Stage


## Step 1 : Elimination Algorithm

\begin{align*}
A_{ij} &\leftarrow A_{ij} - \frac{A_{ik}}{A_{kk}} A_{kj}, \quad j=k, \, k+1, \, \ldots, \, n\\
b_i &\leftarrow b_i - \frac{A_{ik}}{A_{kk}} b_{k},
\end{align*}

for all $i\ge k+1$. 

## Elimination implementation
Code that takes a matrix $A$ and a vector $\boldsymbol{b}$ and converts it into upper-triangular form . It is a form of in-place elimination

In [3]:
def upper_triangle(A, b):
    """ A function to covert A into upper triangluar form through row operations.
    The same row operations are performed on the vector b.
    
    Note that this implementation does not use partial pivoting which is introduced below.
    
    Also note that A and b are overwritten, and hence we do not need to return anything
    from the function.
    """
    n = np.size(b)
    rows, cols = np.shape(A)
    # check A is square
    assert(rows == cols)
    # and check A has the same numner of rows as the size of the vector b
    assert(rows == n)

    # Loop over each pivot row - all but the last row which we will never need to use as a pivot
    for k in range(n-1):
        # Loop over each row below the pivot row, including the last row which we do need to update
        for i in range(k+1, n):
            # Define the scaling factor for this row outside the innermost loop otherwise 
            # its value gets changed as you over-write A!!
            # There's also a performance saving from not recomputing things when not strictly necessary
            s = (A[i, k] / A[k, k])
            # Update the current row of A by looping over the column j
            # start the loop from k as we can assume the entries before this are already zero
            for j in range(k, n):
                A[i, j] = A[i, j] - s*A[k, j]
            # and update the corresponding entry of b
            b[i] = b[i] - s*b[k]


# Test our code on our 2x2 and 3x3 examples from above

# 2x2 example from above
A = np.array([[2., 3.], [1., -4.]])
b = np.array([7., 3.])

upper_triangle(A, b)

print('Our A matrix following row operations to transform it into upper-triangular form:')
pprint(A)
print('The correspondingly updated b vector:')
pprint(b)

# 3x3 example from homework exercise
A = np.array([[2., 3., -4.],
              [6., 8., 2.],
              [4., 8., -6.]])
b = np.array([5., 3., 19.])

upper_triangle(A, b)

print('\nOur A matrix following row operations to transform it into upper-triangular form:')
pprint(A)
print('The correspondingly updated b vector:')
pprint(b)

Our A matrix following row operations to transform it into upper-triangular form:
array([[ 2. ,  3. ],
       [ 0. , -5.5]])
The correspondingly updated b vector:
array([ 7. , -0.5])

Our A matrix following row operations to transform it into upper-triangular form:
array([[ 2.,  3., -4.],
       [ 0., -1., 14.],
       [ 0.,  0., 30.]])
The correspondingly updated b vector:
array([  5., -12., -15.])


### Comments

- Note that in this implementation we operated on $A$ and $b$ independently. 


- We could have saved a few lines of code by forming the augmented matrix inside the function and updating the `for j in range(k, n):` loop to `range(k, n+1)`. 


- `numpy.hstack` could have been be used to form the augmented matrix - you could try implementing this yourself.

## Step 2: Back substitution algorithm
\begin{align*}
 x_{k} &= \frac{1}{A_{kk}}\left( b_k - \sum_{j=k+1}^{n}A_{kj}x_j\right).
\end{align*}

In [4]:
# This function assumes that A is already an upper triangular matrix, 
# e.g. we have already run our upper_triangular function if needed.

def back_substitution(A, b):
    """ Function to perform back subsitution on the system Ax=b.
    
    Returns the solution x.
    
    Assumes that A is on upper triangluar form.
    """
    n = np.size(b)
    # Check A is square and its number of rows and columns same as size of the vector b
    rows, cols = np.shape(A)
    assert(rows == cols)
    assert(rows == n)
    # We can/should check that A is upper triangular using np.triu which is the 
    # upper triangular part of a matrix - if A is already upper triangular, then
    # it should of course match the upper-triangular component of A!!
    assert(np.allclose(A, np.triu(A)))
    
    x = np.zeros(n)
    # start at the end (row n-1) and work backwards
    for k in range(n-1, -1, -1):
        # note that we could do this update in a single vectorised line 
        # using np.dot or @ - this could also speed things up
        s = 0.
        for j in range(k+1, n):
            s = s + A[k, j]*x[j]
        x[k] = (b[k] - s)/A[k, k]

    return x


# This A is the upper triangular matrix carried forward
# from the Python box above, and b the correspondingly updated b vector.
A = np.array([[ 2.,  3., -4.],
              [ 0., -1., 14.],
              [ 0.,  0., 30.]])
b = np.array([  5., -12., -15.])

# print the solution using our codes
x = back_substitution(A, b)
pprint(x)  

# Reinitialise A and b !
# remember our functions overwrote them
A = np.array([[2., 3., -4.],
                 [6., 8., 2.],
                 [4., 8., -6.]])
b = np.array([5., 3., 19.])

# check our answer against what SciPy gives us by multiplying b by A inverse 
pprint(sl.inv(A) @ b)

print('Success: ', np.allclose(x, sl.inv(A) @ b))

array([-6. ,  5. , -0.5])
array([-5.999999999999998 ,  4.999999999999999 , -0.4999999999999998])
Success:  True


## Putting things together - a Gaussian elimination function

As we shall see in the homework, we can wrap up our calls to our upper triangle and back substitution functions with something like

```Python
def gaussian_elimination(A,b):
    Acopy = A.copy()
    bcopy = b.copy()
    upper_triangle(Acopy,bcopy)
    x = back_substitution(Acopy, bcopy)
    return x  
```